In [2]:
import numpy as np
import sklearn as skl
import pandas as pd
import sklearn.preprocessing
from sklearn import linear_model
from sklearn.metrics import f1_score
from scipy import special
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
import keras
from keras.layers import Dense, BatchNormalization
from keras.models import Sequential
from tensorflow.python.client import device_lib

np.random.seed(69) #fixing seed for reproducability

In [12]:
#print(device_lib.list_local_devices())

df_train = pd.read_csv("train.csv")
df_splitt = df_train['Sequence'].apply(lambda x: pd.Series(list(x))) #splitts the 4 mutators into individual columns.
df_active = df_train['Active']
#df_letters = pd.concat([df_train['Sequence'].apply(lambda x: pd.Series(list(x))), df_train['Active']],axis=1)

In [4]:
df_test = pd.read_csv("test.csv")
df_test_s = df_test['Sequence'].apply(lambda x: pd.Series(list(x)))

In [5]:
# define universe of possible input values
mutators = 'ACDEFGHIKLMNPQRSTVWY' #abcdefghijklmnopqrstuvwxyz
#bjoquxz <-- these letters are missing

# define a mapping of chars to integers
char_to_int = dict((c, i) for i, c in enumerate(mutators))
int_to_char = dict((i, c) for i, c in enumerate(mutators))

# integer encode input data
#integer_encoded = [char_to_int[item] for item in df_letters[0]]
#print(integer_encoded)

In [6]:
#Defining some constants.

NUM_MUTATORS = len(mutators)

DF_TRAIN_SIZE = len(df_train.index)
DF_TEST_SIZE = len(df_test.index)

In [16]:
df = np.zeros((DF_TRAIN_SIZE, NUM_MUTATORS*4))

df_occurency = np.zeros((NUM_MUTATORS*4, 1))
#Possibly the worst fucking way we could have done this. But fuckem.
for i in range(0,4):
    for j in range(0, DF_TRAIN_SIZE):
        temp = char_to_int[df_splitt.loc[j,i]]
        df[j, i * NUM_MUTATORS + temp] = 1      #binary representation of the letters
        #df[j, i] = temp             #letter as float (0,1]
        if df_active[j] == 1:
            #Did the mutator at that spot actually lead to a activation? 
            df_occurency[i* NUM_MUTATORS + temp] += 1



In [17]:
#Do the same with the test dataframe

df_cock = np.zeros((DF_TEST_SIZE, NUM_MUTATORS*4))

for i in range(0,4):
    for j in range(0, DF_TEST_SIZE):
        temp = char_to_int[df_test_s.loc[j,i]]
        df_cock[j, i * NUM_MUTATORS + temp] = 1
        #df_cock[j, i] = temp

In [27]:
#Total number of mutators at that spot
df_sum = df.sum(axis=0)
df_probability = np.zeros((NUM_MUTATORS*4, 1))
for i in range(0,NUM_MUTATORS * 4):
    df_probability[i] = df_occurency[i]/df_sum[i]


In [29]:
df_train_prob = np.zeros((DF_TRAIN_SIZE, NUM_MUTATORS*4))
df_test_prob = np.zeros((DF_TEST_SIZE, NUM_MUTATORS*4))

#Possibly the worst fucking way we could have done this. But fuckem.
for i in range(0,4):
    for j in range(0, DF_TRAIN_SIZE):
        temp = char_to_int[df_splitt.loc[j,i]]
        df_train_prob[j, i * NUM_MUTATORS + temp] = df_probability[i* NUM_MUTATORS + temp]      #binary representation of the letters converted to probability of activation
        #df[j, i] = temp             #letter as float (0,1]

for i in range(0,4):
    for j in range(0, DF_TEST_SIZE):
        temp = char_to_int[df_test_s.loc[j,i]]
        df_test_prob[j, i * NUM_MUTATORS + temp] = df_probability[i* NUM_MUTATORS + temp]
        #df_cock[j, i] = temp

In [33]:
#clf = skl.linear_model.RidgeCV(alphas=[0.01, 0.01, 0.1, 1, 10, 100, 1000, 2000,5000], cv=15)

labels = df_train['Active'].to_numpy()

#predict cocks:
#clf.fit(df,labels)
print('Done with fitting')
#predict_train = clf.predict(df)
#print('Done with prediction and error is: ', np.sqrt(np.mean((predict_train-labels)**2)))
#predict_cancer = clf.predict(df_cock)
#predict_cancer = np.where(predict_cancer > 0.5, 1, 0)

Done with fitting


In [79]:
#clf = skl.svm.LinearSVC(dual=False, class_weight='balanced')

#labels = df_train['Active'].to_numpy()

#predict cocks:
#clf.fit(df,labels)
print('Done with fitting')
#predict_train = clf.predict(df)
#print('Done with prediction and error is: ', np.sqrt(np.mean((predict_train-labels)**2)))
#predict_cancer = clf.predict(df_cock)
#predict_cancer = np.where(predict_cancer > 0.5, 1, 0)

Done with fitting


In [80]:
#np.savetxt('submission_sklearn.csv', predict_cancer, delimiter=',', fmt='%i')

In [31]:
#lets norm this shit (this is only necessary if we use the 4D representation
#df_nn = (df+1)/21
#df_cock_nn = (df_cock+1)/21

#for the binary version use this:
df_nn = df
df_cock_nn = df_cock

#print(df_nn.max(), df_nn.min())
#print(labels.max(), labels.min())
#print(df_cock_nn.max(), df_cock_nn.min())

In [36]:
#lets construct the neural net:
neuralNetwork = Sequential()
neuralNetwork.add(Dense(128, activation='relu', input_dim=80))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dense(128, activation='relu'))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dense(128, activation='relu'))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dense(128, activation='relu'))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dense(128, activation='relu'))
neuralNetwork.add(BatchNormalization())
neuralNetwork.add(Dense(1, activation='sigmoid'))
#neuralNetwork.add(BatchNormalization())
neuralNetwork.compile(loss = keras.losses.BinaryCrossentropy(), optimizer = 'adam', metrics=['accuracy'])

#fit the network to (for now the un-sparse matrix)
neuralNetwork.fit(df_nn, labels, epochs=50, batch_size=1024, verbose=0)  #<-- OLD

#evaluation of the network prediction
predict_cancer_nn = neuralNetwork.predict(df_cock_nn)
predict_train_nn = neuralNetwork.predict(df_nn)
print(predict_cancer_nn)
print(predict_train_nn)
predict_cancer_nn = (predict_cancer_nn >= 0.5)

[[3.1055799e-06]
 [2.1041219e-06]
 [2.2878929e-07]
 ...
 [1.3161004e-03]
 [1.3093959e-05]
 [7.8472793e-03]]
[[3.8820676e-06]
 [1.0311024e-05]
 [1.0347408e-06]
 ...
 [1.3235198e-05]
 [1.8877844e-07]
 [1.1063612e-05]]


In [38]:
#fit the network to (for now the un-sparse matrix)
neuralNetwork.fit(df_train_prob, labels, epochs=50, batch_size=1024, verbose=2)  #<-- NEW

#evaluation of the network prediction
predict_testp_nn = neuralNetwork.predict(df_test_prob)
predict_trainp_nn = neuralNetwork.predict(df_train_prob)
print(predict_testp_nn)
print(predict_trainp_nn)
predict_testp_nn = (predict_testp_nn >= 0.5)

Epoch 1/50
110/110 - 1s - loss: 0.0162 - accuracy: 0.9939
Epoch 2/50
110/110 - 1s - loss: 0.0163 - accuracy: 0.9938
Epoch 3/50
110/110 - 1s - loss: 0.0162 - accuracy: 0.9939
Epoch 4/50
110/110 - 1s - loss: 0.0134 - accuracy: 0.9949
Epoch 5/50
110/110 - 1s - loss: 0.0136 - accuracy: 0.9949
Epoch 6/50
110/110 - 1s - loss: 0.0125 - accuracy: 0.9951
Epoch 7/50
110/110 - 1s - loss: 0.0125 - accuracy: 0.9952
Epoch 8/50
110/110 - 1s - loss: 0.0127 - accuracy: 0.9950
Epoch 9/50
110/110 - 1s - loss: 0.0114 - accuracy: 0.9956
Epoch 10/50
110/110 - 1s - loss: 0.0116 - accuracy: 0.9953
Epoch 11/50
110/110 - 1s - loss: 0.0108 - accuracy: 0.9957
Epoch 12/50
110/110 - 1s - loss: 0.0099 - accuracy: 0.9964
Epoch 13/50
110/110 - 1s - loss: 0.0094 - accuracy: 0.9964
Epoch 14/50
110/110 - 1s - loss: 0.0097 - accuracy: 0.9963
Epoch 15/50
110/110 - 1s - loss: 0.0100 - accuracy: 0.9961
Epoch 16/50
110/110 - 1s - loss: 0.0094 - accuracy: 0.9963
Epoch 17/50
110/110 - 1s - loss: 0.0088 - accuracy: 0.9967
Epoch 

In [126]:
print(predict_train_nn.max())
print(predict_train_nn.min())

print("actual labels:", np.sum(labels))
print("predicted labels:", np.sum(predict_train_nn >0.5))
print("the f1 score is:", f1_score(labels.astype('bool'), (predict_train_nn >=0.5).astype('bool')))

1.0
1.3374322e-13
actual labels: 4213
predicted labels: 4224
the f1 score is: 0.9979850657816759


In [127]:
predict_cancer_nn = pd.DataFrame(predict_cancer_nn.astype('int'))
predict_cancer_nn.to_csv('predictions.csv', header = False, index = False)


In [39]:
#New
print(predict_trainp_nn.max())
print(predict_trainp_nn.min())

print("actual labels:", np.sum(labels))
print("predicted labels:", np.sum(predict_trainp_nn >0.5))
print("the f1 score is:", f1_score(labels.astype('bool'), (predict_trainp_nn >=0.5).astype('bool')))

predict_testp_nn = pd.DataFrame(predict_testp_nn.astype('int'))
predict_testp_nn.to_csv('predictions_with_probability.csv', header = False, index = False)


1.0
1.1738408e-17
actual labels: 4213
predicted labels: 4029
the f1 score is: 0.9728221305508372
